In [ ]:
import sys
sys.path.append('..')
import pandas as pd
from Constants import Datasets
import numpy as np
datasets = [dataset.name for dataset in Datasets]
metrics = [
'total_num_words',
'avg_word_length', 
'total_num_sent',
'avg_sent_length',
'token_type_ratio',
'symbol_word_ratio',
'num_non_alphabet_words'
]
num_samples = 10000

In [ ]:
#embeddings dim reduced
points = pd.DataFrame()
for i, d1 in enumerate(datasets):
    d = pd.DataFrame(np.random.randn(num_samples, 2) - i, columns=['pc1', 'pc2'])
    d['name'] = d1
    points = pd.concat((points, d))
import uuid
points['id'] = [uuid.uuid4() for _ in range(len(points))]

In [ ]:
#generate dummy metrics for datasets
all_metrics = pd.DataFrame()
for j, dataset in enumerate(datasets):
    metrics_for_dataset = pd.DataFrame(np.random.randn(num_samples, len(metrics)) + j, columns=metrics)
    all_metrics = pd.concat((all_metrics, metrics_for_dataset))
all_metrics['id'] = points['id']

In [ ]:
points.to_csv('../datasets/dataset.csv', index=False)
all_metrics.to_csv('../datasets/metrics.csv', index=False)